# Let's compile everything that we've learned so far

Let's try to learn other machine learning model pipeline called "Table question answering" which is a question answering system that answers questions about tables. The best way to understand it is to just refer to below code and run it:


In [ ]:
%pip install transformers
%pip install pandas

In [1]:
from transformers import pipeline

data = [
    {
        "House": "House 1",
        "City": "Jakarta",
        "Price (in million IDR)": "5000",
        "Rooms": "2 bed, 2 bath",
        "Facilities": "Pool, Garage, Gym, Garden",
        "Furnished": "Yes",
    },
    {
        "House": "House 2",
        "City": "Surabaya",
        "Price (in million IDR)": "2000",
        "Rooms": "3 bed, 2 bath",
        "Facilities": "Garage, Library",
        "Furnished": "No",
    },
    {
        "House": "House 3",
        "City": "Malang",
        "Price (in million IDR)": "1500",
        "Rooms": "2 bed, 1 bath",
        "Facilities": "Pool, Gym, Sauna",
        "Furnished": "No",
    },
    {
        "House": "House 4",
        "City": "Jakarta",
        "Price (in million IDR)": "5300",
        "Rooms": "1 bed, 1 bath",
        "Facilities": "Gym, Rooftop",
        "Furnished": "Yes",
    },
    {
        "House": "House 5",
        "City": "Surabaya",
        "Price (in million IDR)": "2200",
        "Rooms": "3 bed, 2 bath",
        "Facilities": "Pool, Garage, Gym, Library",
        "Furnished": "No",
    },
    {
        "House": "House 6",
        "City": "Malang",
        "Price (in million IDR)": "1600",
        "Rooms": "2 bed, 2 bath",
        "Facilities": "Garage, Gym, Garden",
        "Furnished": "Yes",
    },
    {
        "House": "House 7",
        "City": "Jakarta",
        "Price (in million IDR)": "4900",
        "Rooms": "4 bed, 3 bath",
        "Facilities": "Pool, Garage, Sauna",
        "Furnished": "Yes",
    },
    {
        "House": "House 8",
        "City": "Surabaya",
        "Price (in million IDR)": "2100",
        "Rooms": "3 bed, 3 bath",
        "Facilities": "Gym, Garden",
        "Furnished": "No",
    },
    {
        "House": "House 9",
        "City": "Malang",
        "Price (in million IDR)": "1400",
        "Rooms": "4 bed, 2 bath",
        "Facilities": "Garage, Rooftop",
        "Furnished": "Yes",
    },
    {
        "House": "House 10",
        "City": "Jakarta",
        "Price (in million IDR)": "5100",
        "Rooms": "3 bed, 1 bath",
        "Facilities": "Pool, Garden, Sauna",
        "Furnished": "No",
    },
]

# Set up the pipeline:

table_qa = pipeline("table-question-answering", model="google/tapas-large-finetuned-wtq")


# Ask a question:

query = "Which house is the most expensive?"
table_qa({"table": data, "query": query})

D:\Repositories\skill-academy-ml\venv\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:2762: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
D:\Repositories\skill-academy-ml\venv\lib\site-packages\transformers\models\tapas\tokenization_tapas.py:1561: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


{'answer': 'House 4',
 'coordinates': [(3, 0)],
 'cells': ['House 4'],
 'aggregator': 'NONE'}

As you can see, "TableQA" pipeline is basically when we want to ask a machine learning model a question about a set of data. Above table is about several dummy data on houses in several cities.

This pipeline will output different things based on given question. Try below questions:

- What is the average price of houses in Jakarta?
- I want to buy house 7 and house 10, what's the total price?
- How many houses in Surabaya that has gym as one of the facility?

# Quick Note: Tuple

Tuple is a data structure that is similar to list, but it is immutable. It means that you cannot change the value of tuple once it is created (we cannot mutate any of its value). You'll see tuple on the output of coordinates from above code.

The syntax of tuple is similar to list, but instead of using square brackets, we use round brackets:

In [2]:
print((5, 0))
print((5, 0)[0])
(5, 0)[0] = 3 # Error

(5, 0)
5


TypeError: 'tuple' object does not support item assignment

# Different question can lead to different output

We're using `TAPAS` model by Google, and it's been fine-tuned on WTQ dataset. Unfortunately we can't dive deeper into learning what each of that mean because it's outside the context of our learning, but check below image:

![Tapas for QnA](https://storage.googleapis.com/rg-ai-bootcamp/python-prep/tapas-for-qa-min.png)

> This case is also called weak supervision, since the model itself must learn the appropriate aggregation operator (SUM/COUNT/AVERAGE/NONE) given only the answer to the question as supervision.

Above image and snippet can be found at https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/tapas#transformers.TFTapasForSequenceClassification, feel free to read it if you're interested (Warning: it's quite technical). To summarize basically TAPAS fine-tuned with WTQ has 4 different output:

- `NONE`: No aggregation is needed, the answer is a single cell.
- `SUM`: We should sum the values given in the output.
- `COUNT`: We should count the number of values given in the output.
- `AVERAGE`: We should average the values given in the output.

# Challenge: Start with `count`

Let's start with creating the functionality to aggregate `count` first. One of the question that will be answered with `count` aggregation is: "How many houses in Surabaya that has gym as one of the facility?". The output can be seen below:

```python
{'answer': 'COUNT > House 5, House 6, House 8',
 'coordinates': [(4, 0), (5, 0), (7, 0)],
 'cells': ['House 5', 'House 6', 'House 8'],
 'aggregator': 'COUNT'}
```

We'll create two different function for now:

- `aggregate_data`: This function should input above dictionary, and will choose which aggregator to use based on the `aggregator` key. It will return the aggregated value.
- `count_aggregator`: This function should input above dictionary, and will count the number of cells in `cells` key. It will return the total count.

So you might have to realize that:

- For `none` aggregator, you don't need aggregator, just return the `answer` key as it is.
- For `count` aggregator, you need to call `count_aggregator` inside the `aggregate_data` function and return the result of `count_aggregator` function.

In [11]:
def aggregate_data(data):
    match data['aggregator']:
        case "COUNT":
            return count_aggregator(data)
        case _:
            return data

def count_aggregator(data):
    return len(data["cells"])  # Your answer here

aggregate_data({'answer': 'COUNT > House 5, House 6, House 8',
 'coordinates': [( 4, 0), (5, 0), (7, 0)],
 'cells': ['House 5', 'House 6', 'House 8'],
 'aggregator': 'COUNT'})

3

When you are done with the above challenge, then:

1. Run the code block by pressing the play button.

In [3]:
!pip install rggrader

from rggrader import submit

# @title #### Student Identity
student_id = "REA3X5EN" # @param {type:"string"}
name = "Steven Adi Santoso" # @param {type:"string"}

# Submit Method
assignment_id = "013_practical_example_tableqa_part_1"
question_id = "01_count_aggregator"

data_count = {'answer': 'COUNT > House 1, House 2',
 'coordinates': [( 4, 0), (5, 0), (7, 0)],
 'cells': ['House 1', 'House 2'],
 'aggregator': 'COUNT'}

submit(student_id, name, assignment_id, str(count_aggregator(data_count)), question_id)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


'Assignment successfully submitted'

# Challenge: Create `sum` and `average` aggregator

For `sum` and `average` aggregator, you need to use the `cells` key again, convert the `string` to `int`, and then `sum` or `average` it. Feel free to use `for loop` or `map` function to do it.

In [4]:
def sum_aggregator(data):
    return sum(map(int, data['cells'])) # Your answer here

def average_aggregator(data):
    return sum(map(int, data['cells'])) / len(data['cells']) # Your answer here

# Make sure to add additional aggregators to below function
def aggregate_data(data):
    match data['aggregator']:
        case "COUNT":
            return count_aggregator(data)
        case "SUM":
            return sum_aggregator(data)
        case "AVERAGE":
            return average_aggregator(data)
        case _:
            return data

print(aggregate_data({'answer': 'SUM > 4900, 5100',
 'coordinates': [(6, 2), (9, 2)],
 'cells': ['4900', '5100'],
 'aggregator': 'SUM'}))

print(aggregate_data({'answer': 'AVERAGE > 5000, 5300, 4900, 5100',
 'coordinates': [(0, 2), (3, 2), (6, 2), (9, 2)],
 'cells': ['5000', '5300', '4900', '5100'],
 'aggregator': 'AVERAGE'}))

10000
5075.0


When you are done with the above challenge, then:

1. Run the code block by pressing the play button.

In [5]:
# Submit Method
data_sum = {'answer': 'SUM > 4900, 5300',
 'coordinates': [(6, 2), (9, 2)],
 'cells': ['4900', '5300'],
 'aggregator': 'SUM'}

assignment_id = "013_practical_example_tableqa_part_1"
question_id = "02_sum_aggregator"
submit(student_id, name, assignment_id, str(sum_aggregator(data_sum)), question_id)

#####################################################################################
# Submit Method
data_avg = {'answer': 'AVERAGE > 5000, 2000, 1500, 5300',
 'coordinates': [(0, 2), (3, 2), (6, 2), (9, 2)],
 'cells': ['5000', '2000', '1500', '5300'],
 'aggregator': 'AVERAGE'}

assignment_id = "013_practical_example_tableqa_part_1"
question_id = "03_avg_aggregator"
submit(student_id, name, assignment_id, str(average_aggregator(data_avg)), question_id)

'Assignment successfully submitted'

# Challenge: Connecting with the pipeline

We've created the full functionality for what we need, now we just need to connect it with the pipeline. Now let's see if you can try to connect previous `aggregate_data` function with `table_qa` function!

In [12]:
# Let's try to connect `table_qa` with `aggregate_data`:

def answer_table_question(question):
    return aggregate_data(table_qa({"table": data, "query": question})) # Your answer here

answer_table_question("What is the average price of the houses in Jakarta?") # The answer should be 5075
answer_table_question("How many houses are there in Malang?") # The answer should be 3
answer_table_question("How many houses in Jakarta has a pool as a facility?") # The answer should be 3

3

When you are done with the above challenge, then:

1. Run the code block by pressing the play button.

In [13]:
# Submit Method
assignment_id = "013_practical_example_tableqa_part_1"
question_id = "04_answer_table"
submit(student_id, name, assignment_id, str(answer_table_question("What is the average price of the houses in Surabaya?")), question_id)

'Assignment successfully submitted'